In [ ]:
import os
import json
import numpy
import pyccl
import scipy

In [ ]:
# FOLDER
FOLDER = '/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/LimberCloud/'
INFO_FOLDER = os.path.join(FOLDER, 'INFO')

In [ ]:
# Cosmology
with open(os.path.join(INFO_FOLDER, 'COSMOLOGY.json'), 'r') as file:
    COSMOLOGY = json.load(file)

CCL = pyccl.Cosmology(
    h = COSMOLOGY['H'],
    w0 = COSMOLOGY['W0'],
    wa = COSMOLOGY['WA'], 
    n_s = COSMOLOGY['NS'], 
    A_s = COSMOLOGY['AS'],
    m_nu = COSMOLOGY['M_NU'],  
    Neff = COSMOLOGY['N_EFF'],
    Omega_b = COSMOLOGY['OMEGA_B'], 
    Omega_k = COSMOLOGY['OMEGA_K'], 
    Omega_c = COSMOLOGY['OMEGA_CDM'], 
    mass_split = 'single', matter_power_spectrum = 'halofit', transfer_function = 'boltzmann_camb',
    extra_parameters = {'camb': {'kmax': 50, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [ ]:
# Comparison
Z1 = 0.0
Z2 = 3.5
GRID_SIZE = 350
Z_GRID = numpy.linspace(Z1, Z2, GRID_SIZE + 1)

A_GRID = 1 / (1 + Z_GRID)
CHI_GRID = pyccl.background.comoving_radial_distance(cosmo=CCL, a=A_GRID)

CHI_SIZE = 500
Z_DATA = numpy.linspace(Z1, Z2, CHI_SIZE + 1)

A_DATA = 1 / (1 + Z_DATA)
CHI_DATA = pyccl.background.comoving_radial_distance(cosmo=CCL, a=A_DATA)

ELL1 = 20
ELL2 = 2000
ELL_SIZE = 20
ELL_GRID = numpy.geomspace(ELL1, ELL2, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_GRID)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out=numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where=CHI_MESH > 0))

POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):
    POWER_GRID[:,GRID_INDEX] = pyccl.power.nonlin_matter_power(cosmo=CCL, k=SCALE_GRID[:,GRID_INDEX], a=A_GRID[GRID_INDEX])

In [ ]:
# Numerical integral

def integral_I9(chi1, chi2, chi3, chi4, chi5, chi6, chi7, power1, power2, redshift1, redshift2):
    
    def formula_0(chi):
        result = ((chi5 - chi3) / 2 - chi * chi5 * numpy.log(chi5 / chi4) / (chi5 - chi4) + chi * chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3))
        
        result = result * (- chi + (chi7 - chi6) / 2 + chi * chi6 * numpy.log(chi7 / chi6) / (chi7 - chi6))
        
        result = result * (((chi / chi2) ** 3) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    def formula_n(chi):
        result = ((chi5 - chi3) / 2 - chi * chi5 * numpy.log(chi5 / chi4) / (chi5 - chi4) + chi * chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3))
        
        result = result * (- chi + (chi7 - chi6) / 2 + chi * chi6 * numpy.log(chi7 / chi6) / (chi7 - chi6))
        
        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    if chi1 == 0:
        integral = scipy.integrate.quad_vec(f=formula_0, a=chi1, b=chi2)[0]
    else:
        integral = scipy.integrate.quad_vec(f=formula_n, a=chi1, b=chi2)[0]
    return integral

In [ ]:
# Coefficient

def coefficient_J9(chi1, chi2, chi3, chi4, chi5, chi6, chi7, power1, power2, redshift1, redshift2):
    a = 1 - chi1 / chi2
    b = (chi5 - chi3) / (2 * chi2)
    c = chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3) - chi5 * numpy.log(chi5 / chi4) / (chi5 - chi4)
    d = (chi7 - chi6) / (2 * chi2)
    e = chi6 * numpy.log(chi7 / chi6) / (chi7 - chi6) - 1
    y = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)
    
    if a == 1:
        formula = (1 / 840) * (5 * c * e * (28 + ( - 8 + z) * z) + 8 * c * d * (21 + ( - 7 + z) * z) + 8 * b * e * (21 + ( - 7 + z) * z) + 14 * b * d * (15 + ( - 6 + z) * z))
    else:
        formula = (1 / 60) * a * (b * ( - 30 * e * ( - 2 + y + 2 * z) + 5 * e * z * (y * (8 - 3 * z) + 4 * z) + 20 * d * (3 + ( - 3 + z) * z) + a * e * ( - 30 + 20 * y + 5 * (8 - 3 * z) * z + 6 * y * z * ( - 5 + 2 * z)) - 
5 * d * y * (6 + z * ( - 8 + 3 * z))) + c * (e * (20 * (3 + ( - 3 + a) * a) - 5 * (6 + a * ( - 8 + 3 * a)) * y + 2 * ( - 30 + 5 * (8 - 3 * a) * a + 20 * y + 6 * a * ( - 5 + 2 * a) * y) * z + 
(20 - 15 * y + 2 * a * ( - 15 + 6 * a + 12 * y - 5 * a * y)) * z ** 2) + d * (20 * (3 + ( - 3 + z) * z) + a * ( - 30 + 20 * y + 5 * (8 - 3 * z) * z + 6 * y * z * ( - 5 + 2 * z)) - 
5 * y * (6 + z * ( - 8 + 3 * z)))))
    
    coefficient = chi2 ** 3 * power2 * (1 + redshift2) ** 2 * formula
    return coefficient

In [ ]:
# Case1: chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]
CHI4 = CHI_GRID[GRID_INDEX + 3]
CHI5 = CHI_GRID[GRID_INDEX + 4]
CHI6 = CHI_GRID[GRID_SIZE - 1]
CHI7 = CHI_GRID[GRID_SIZE]

POWER1 = POWER_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

1461.30684999374 1497.8183354014277 1534.1133113296617 1570.1927029383162 1606.0570772924168 2616.3612383175455 2645.867010982371 49919.45246058225 48958.62248873209 0.36 0.37


In [ ]:
# Case1: chi1 > 0

INTEGRAL = integral_I9(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J9(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

(45153045.338139, 45153045.33813212, -1.5232259897857148e-13)

In [ ]:
# Case2: chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]
CHI4 = CHI_GRID[GRID_INDEX + 3]
CHI5 = CHI_GRID[GRID_INDEX + 4]
CHI6 = CHI_GRID[GRID_SIZE - 1]
CHI7 = CHI_GRID[GRID_SIZE]

POWER1 = POWER_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.42674493673601 88.64121385461215 132.64235496422637 176.42911660960908 2616.3612383175455 2645.867010982371 0.0 2114.2567965927065 0.0 0.01


In [ ]:
# Case2: chi1 = 0

INTEGRAL = integral_I9(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J9(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

(11077285.177621555, 11077285.177621556, 2.220446049250313e-16)